# Capston Project Notebook

## Assignment 0: Hello world

This notebook will be used mainly for the Coursera Capstone Project

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Assignment 1: Toronto neighborhoods 

In [3]:
import pandas as pd
import re

Reading the table of Toronto postal codes from the wikipedia htlm page with the pandas.read_html function

In [4]:
dfs=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header=0)
postal_code_df=dfs[0]

1. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [5]:
postal_code_df.drop(index=postal_code_df[postal_code_df['Borough']=='Not assigned'].index, inplace=True)
postal_code_df.reset_index(drop=True, inplace=True)
postal_code_df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


2. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.

This task is obtained by means of regular expressions

In [6]:
new_neighborhoods=[]
for neighbor in postal_code_df['Neighborhood']:
    new_neighborhoods.append(re.sub(' /', ',', neighbor))

postal_code_df['Neighborhood']=new_neighborhoods
postal_code_df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


3. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [7]:
postal_code_df['Neighborhood'][postal_code_df['Neighborhood']=='Not assigned']=postal_code_df['Borough'][postal_code_df['Neighborhood']=='Not assigned']
postal_code_df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


4. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [8]:
print('The postal code dataframe contains:', postal_code_df.shape[0], 'rows')

The postal code dataframe contains: 103 rows
